<a href="https://colab.research.google.com/github/vaneesa-writes/micro_project/blob/main/face_recog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
!kaggle datasets download -d pramod722445/face-recognition-30

face-recognition-30.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile
zf = "/content/face-recognition-30.zip"
target_dir = "/content/input/face-recognition-30/dataset"
zfile = zipfile.ZipFile(zf)
zfile.extractall(target_dir)

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib as mpl

# style your matplotlib
mpl.style.use("seaborn-darkgrid")
# run this block

In [6]:
from tqdm import tqdm

In [7]:
files=os.listdir("/content/input/face-recognition-30/dataset/dataset/")
files


['Johnny_Galeck',
 'Courteney_Cox',
 'Jennifer_Aniston',
 'aishwarya_rai',
 'hardik_pandya',
 'manoj_bajpayee',
 'Jim_Parsons',
 'angelina_jolie',
 'David_Schwimmer',
 'random_person',
 'scarlett_johansson',
 'sylvester_stallone',
 'mohamed_ali',
 'Lisa_Kudrow',
 'ronaldo',
 'Simon_Helberg',
 'Matthew_Perry',
 'Pankaj_Tripathi',
 'pewdiepie',
 'ROHIT_SHARMA',
 'dhoni',
 'Sachin_Tendulka',
 'suresh_raina',
 'brad_pitt',
 'bhuvan_bam',
 'messi',
 'virat_kohli',
 'arnold_schwarzenegger',
 'Kunal_Nayya',
 'Matt_LeBlanc']

In [8]:
index_2_name={}
name_2_index={}

In [9]:
image_array=[]  # it's a list later i will convert it to array
label_array=[]
path="/content/input/face-recognition-30/dataset/dataset/"
# loop through each sub-folder in train
for i in range(len(files)):
    # files in sub-folder
    file_sub=os.listdir(path+files[i])
    index_2_name[i]=files[i]
    name_2_index[files[i]]=i

    for k in tqdm(range(len(file_sub))):
        try:
            img=cv2.imread(path+files[i]+"/"+file_sub[k])
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img=cv2.resize(img,(96,96))
            image_array.append(img)
            label_array.append(i)
        except:
            pass

100%|██████████| 449/449 [00:01<00:00, 382.35it/s]


In [10]:
print(index_2_name)
print()
print(name_2_index)

{0: 'Johnny_Galeck', 1: 'Courteney_Cox', 2: 'Jennifer_Aniston', 3: 'aishwarya_rai', 4: 'hardik_pandya', 5: 'manoj_bajpayee', 6: 'Jim_Parsons', 7: 'angelina_jolie', 8: 'David_Schwimmer', 9: 'random_person', 10: 'scarlett_johansson', 11: 'sylvester_stallone', 12: 'mohamed_ali', 13: 'Lisa_Kudrow', 14: 'ronaldo', 15: 'Simon_Helberg', 16: 'Matthew_Perry', 17: 'Pankaj_Tripathi', 18: 'pewdiepie', 19: 'ROHIT_SHARMA', 20: 'dhoni', 21: 'Sachin_Tendulka', 22: 'suresh_raina', 23: 'brad_pitt', 24: 'bhuvan_bam', 25: 'messi', 26: 'virat_kohli', 27: 'arnold_schwarzenegger', 28: 'Kunal_Nayya', 29: 'Matt_LeBlanc'}

{'Johnny_Galeck': 0, 'Courteney_Cox': 1, 'Jennifer_Aniston': 2, 'aishwarya_rai': 3, 'hardik_pandya': 4, 'manoj_bajpayee': 5, 'Jim_Parsons': 6, 'angelina_jolie': 7, 'David_Schwimmer': 8, 'random_person': 9, 'scarlett_johansson': 10, 'sylvester_stallone': 11, 'mohamed_ali': 12, 'Lisa_Kudrow': 13, 'ronaldo': 14, 'Simon_Helberg': 15, 'Matthew_Perry': 16, 'Pankaj_Tripathi': 17, 'pewdiepie': 18, 'R

In [11]:
import gc
gc.collect()

100

In [12]:
image_array=np.array(image_array)/255.0
label_array=np.array(label_array)

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(image_array,label_array,test_size=0.15)


In [14]:
from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential,Model,load_model

In [15]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
# from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input

In [16]:
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [17]:
model=Sequential()
# I will use MobileNetV2 as an pretrained model 
pretrained_model=MobileNetV2(input_shape=(96,96,3),include_top=False,
                                         weights="imagenet")
model.add(pretrained_model)
model.add(layers.GlobalAveragePooling2D())
# add dropout to increase accuracy by not overfitting
model.add(layers.Dropout(0.3))
# add dense layer as final output
model.add(layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_96 (Functi  (None, 3, 3, 1280)       2257984   
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 2,259,265
Trainable params: 2,225,153
Non-trainable params: 34,112
_________________________________________________________________


In [18]:
model.compile(optimizer="adam",loss="mean_squared_error",metrics=["mae"])


In [19]:
ckp_path="trained_model/model"
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=ckp_path,
                                                   monitor="val_mae",
                                                   mode="auto",
                                                   save_best_only=True,
                                                   save_weights_only=True)

In [20]:
# create a lr reducer which decrease learning rate when accuarcy does not increase
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(factor=0.9,monitor="val_mae",
                                             mode="auto",cooldown=0,
                                             patience=5,verbose=1,min_lr=1e-6)
# patience : wait till 5 epoch
# verbose : show accuracy every 1 epoch
# min_lr=minimum learning rate

In [21]:
EPOCHS=150
BATCH_SIZE=64

history=model.fit(X_train,
                 Y_train,
                 validation_data=(X_test,Y_test),
                 batch_size=BATCH_SIZE,
                 epochs=EPOCHS,
                 callbacks=[model_checkpoint,reduce_lr],
                  steps_per_epoch=128
                  

                 )

Epoch 1/150
128/128 [==============================] - 28s 97ms/step - loss: 46.7098 - mae: 5.0265 - val_loss: 361.7767 - val_mae: 17.8070 - lr: 0.0010
Epoch 2/150
128/128 [==============================] - 11s 89ms/step - loss: 19.4089 - mae: 3.1608 - val_loss: 256.0992 - val_mae: 14.6648 - lr: 0.0010
Epoch 3/150
128/128 [==============================] - 11s 84ms/step - loss: 14.9120 - mae: 2.7004 - val_loss: 218.0982 - val_mae: 13.3380 - lr: 0.0010
Epoch 4/150
128/128 [==============================] - 10s 81ms/step - loss: 11.5581 - mae: 2.3046 - val_loss: 444.4761 - val_mae: 19.8673 - lr: 0.0010
Epoch 5/150
128/128 [==============================] - 11s 85ms/step - loss: 9.6182 - mae: 2.0842 - val_loss: 111.9601 - val_mae: 9.0303 - lr: 0.0010
Epoch 6/150
128/128 [==============================] - 11s 87ms/step - loss: 8.0649 - mae: 1.9066 - val_loss: 160.5097 - val_mae: 11.4594 - lr: 0.0010
Epoch 7/150
128/128 [==============================] - 11s 82ms/step - loss: 8.4344 - mae: 

In [ ]:
prediction_val=model.predict(X_test,batch_size=64)
prediction_val[:20]
for ele in prediction_val[:20]:
  print(ele[0])

In [23]:
# import cv2
 
# # Save image in set directory
# # Read RGB image
# img = cv2.imread('/content/input/face-recognition-30/dataset/dataset/Pankaj_Tripathi/1.jpg')

In [24]:
# img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
# img=cv2.resize(img,(96,96))
# img=img/255.0
# img

In [25]:
# img.shape

In [26]:
# pred=model.predict(img)
# print(pred)

In [27]:
from keras.preprocessing import image

In [57]:
img = image.load_img('/content/input/face-recognition-30/dataset/dataset/Pankaj_Tripathi/45.jpg',target_size=(96,96))

x = image.img_to_array(img)

x=np.expand_dims(x,axis=0)

x=preprocess_input(x)

In [58]:
pred = model.predict(x)
pred
num=round(pred[0][0])
print(index_2_name[num])


ROHIT_SHARMA


In [42]:
!wget https://images.indianexpress.com/2021/09/Pankaj-Tripathi-1200.jpg


--2022-05-22 18:13:27--  https://images.indianexpress.com/2021/09/Pankaj-Tripathi-1200.jpg
Resolving images.indianexpress.com (images.indianexpress.com)... 104.78.233.112
Connecting to images.indianexpress.com (images.indianexpress.com)|104.78.233.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 77145 (75K) [image/jpeg]
Saving to: ‘Pankaj-Tripathi-1200.jpg’

Pankaj-Tripathi-120 100%[===================>]  75.34K  --.-KB/s    in 0.02s   

2022-05-22 18:13:27 (3.61 MB/s) - ‘Pankaj-Tripathi-1200.jpg’ saved [77145/77145]



In [48]:
img = image.load_img('/content/Pankaj-Tripathi-1200.jpg',target_size=(96,96))

x = image.img_to_array(img)

x=np.expand_dims(x,axis=0)

x=preprocess_input(x)
pred = model.predict(x)
# print(pred)
pred
# num=round(pred[0][0])
print(index_2_name[num])

David_Schwimmer
